# Obtaining SEDs of Seyfert galaxies
In this file we obtain the photometry for each of the galaxies. First, we upload the Python packages.

In [1]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [5]:
import astropy.units as u
from astropy import coordinates
from astropy.table import Table, vstack, unique, join

from astroquery.ned import Ned
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
from astroquery.ned.core import RemoteServiceError, TableParseError

import requests
from os import path
from datetime import datetime
from numpy import unique as uniq

## Get the SEDs
We define a class that will get the SEDs from [CDS](https://cds.u-strasbg.fr/) and [NED](https://ned.ipac.caltech.edu/), add the bibcode to the CDS tables and eliminate photometric duplicates

In [6]:
class ObtainPhotometry:
    """This class allows us to download the SEDs from NED and SIMBAD"""
    def __init__(self, Name, Smot=False):
        self.name = Name
        self.tmpCDS = '../Data/Interim/CDSVotables/'
        self.tmpNED = '../Data/Interim/NEDVotables/'
        print('Working on galaxy ', self.name)
        self.ObtainVot()
        self.ReadVotTables()
        self.AddBibcodeCDS()
        self.CheckBothTables()
#         self.PlotSED(Smot) ## Just when we want to see the SED

    def ObtainVot(self):
        """Function to obtain the VOTables"""
        print(datetime.now(), 'Get CDS SED')
        if path.exists(self.tmpCDS+self.name+'.vot'):
            print('Reading CDS file')
        else:
            time.sleep(5) ## This is used to avoid being flag by the server
            GalaxyS = Simbad.query_object(self.name)
            RA = GalaxyS[0]['RA'].replace(' ', '%20')
            DEC = GalaxyS[0]['DEC'].replace(' ', '%20')
            irl = 'http://vizier.u-strasbg.fr/viz-bin/sed?-c=' + \
                str(RA)+'%20'+str(DEC)+'&-c.rs=5'
            print(irl)
            r = requests.get(irl)
            open(self.tmpCDS+self.name+'.vot', 'wb').write(r.content)
        print(datetime.now(), 'Got CDS SED')
        self.NEDFlag = False
        if path.exists(self.tmpNED+self.name+'.vot'):
            print('Reading NED file')
        else:
            #             time.sleep(0.5)
            try:
                NEDTab = Ned.get_table(self.name, table='photometry')
                NEDTab[NEDTab['NED Units'] == b'Jy'].write(self.tmpNED+self.name+'.vot',
                                                           format='votable') # Use just photometry
            except:
                self.NEDFlag = True

    def ReadVotTables(self):
        """Function to read the VOTables"""
        self.CDSTable = Table.read(self.tmpCDS+self.name+'.vot',
                                   format='votable')
        if self.NEDFlag:
            print('No NED Table')
            self.NEDTable = Table(names=['Refcode', 'Flux_Density', 'Observed_Passband',
                                         'Frequency', 'NED_Uncertainty'], masked=True)
        else:
            self.NEDTable = Table.read(self.tmpNED+self.name+'.vot',
                                       format='votable')

    def PlotSED(self, smooth):
        """In case we want to plot the SED"""
        if smooth:
            self.SmoothSED()
            print(self.CDSTable.columns, self.NEDTable.columns)
        scatter(self.CDSTable['sed_freq'].to(u.micron, equivalencies=u.spectral()),
                self.CDSTable['sed_flux'], label='CDS')
        scatter(self.NEDTable['Frequency'].to(u.micron, equivalencies=u.spectral()),
                self.NEDTable['Flux_Density'], marker='*', label='NED')
        loglog()
        xlabel('Wavelength [um]')
        ylabel('Flux [Jy]')
        legend()

    def AddBibcodeCDS(self):
        """Adding the Bibcode to the CDS tables"""
        self.CDSTable['Bibcode'] = np.array(['Empty']*len(self.CDSTable),
                                            dtype='object')
        for tabindx, tabinfo in enumerate(self.CDSTable['_tabname']):
            try:
                Namcat = tabinfo.rpartition('/')[0]
                Search = Vizier.query_constraints(catalog='METAcat', name=Namcat)
                self.CDSTable['Bibcode'][tabindx] = Search[0][0]['bibcode']
            except:
                print('There is an error at ', tabindx, tabinfo)

    def CheckBothTables(self):
        """Check the rows of CDS and NED and remove all duplicates in the photometric bands"""
        ToRem = []
        for URefcode in np.unique(self.NEDTable['Refcode']):
            LCDS = np.where(self.CDSTable['Bibcode'] == URefcode.decode('utf-8'))[0]
            LNED = np.where(self.NEDTable['Refcode'] == URefcode)[0]
            if len(LCDS) > 0 and len(LNED) > 0:
                print('Duplicate!')
                for lcds in LCDS:
                    for lned in LNED:
                        if str(self.NEDTable[lned]['Flux_Density']) == str(self.CDSTable[lcds]['sed_flux']):
                            print('Deleting NED filter ',
                                  self.NEDTable[lned]['Observed_Passband'],
                                  ' with Bibcode ',
                                  self.NEDTable[lned]['Refcode'])
                            ToRem.append(lned)
        self.NEDTable.remove_rows(ToRem)

    def SmoothSED(self):
        """Create a smooth SED, only for plotting"""
        self.CDSTable = self.CDSTable.group_by('sed_freq').groups.aggregate(np.mean)
        self.NEDTable = self.NEDTable.group_by('Frequency').groups.aggregate(np.mean)

Then, we define a class to clean the Photometry and ignore empty values in the tables

In [7]:
class CleanPhotometry:
    """Clean the photometry in CDS and NED tables"""
    def __init__(self, TableCDS, TableNED):
        print('Cleaning')
        self.CDSTable = TableCDS
        self.NEDTable = TableNED
        self.JoinTables()

    def CleanCDSTo(self, table):
        """Remove rows with masked (empty) and null (0) values, then average the values."""
        table.remove_rows(where(table['sed_eflux'].mask)[0])
        table.remove_rows(where(table['sed_eflux'] == 0.0)[0])
        Vales, counts = numpy.unique(table['sed_flux'], return_counts=True)
        if len(Vales) > 0:
            AVG = np.average(Vales, weights=counts)
            STD = np.sqrt(np.sum(table['sed_eflux']**2))
            return(AVG, STD)
        else:
            return(nan, nan)

    def CleanNEDTo(self, table):
        """Remove rows with masked (empty) and string values, then average the values."""
        table.remove_rows(where(table['Flux_Density'].mask)[0])
        table.remove_rows(where(table['NED_Uncertainty'] == b'')[0])
        table.remove_rows(where(table['NED_Uncertainty'] == b'+/-...')[0])
        Vales, counts = numpy.unique(table['Flux_Density'], return_counts=True)
        table['NED_Uncertainty'] = [float(j.split(b'+/-')[-1]) for j in table['NED_Uncertainty']]
        if len(Vales) > 0:
            AVG = np.average(Vales, weights=counts)
            STD = np.sqrt(np.sum(table['NED_Uncertainty']**2))
            return(AVG, STD)
        else:
            return(nan, nan)

    def FiltersNED(self, tableFilter):
        """ Use only the information that we need from NED """
        Tabls = []
        Band = [where(self.NEDTable['Observed_Passband'] == filt)[0] for filt in tableFilter]
        for bandinx, bandif in enumerate(Band):
            AVG, STD = self.CleanNEDTo(self.NEDTable[bandif])
            if isnan(AVG):
                continue
            else:
                Tabls.append([tableFilter[bandinx],
                              np.mean(self.NEDTable[bandif]['Frequency'].to(u.micron,
                                                                            equivalencies=u.spectral())),
                              AVG*u.Jy, STD*u.Jy])
        return(Table(array(Tabls),
                     names=['Filter', 'Wave', 'Flux', 'F_er'],
                     dtype=('U32', 'float64', 'float64', 'float64')))

    def FiltersCDS(self, tableFilter):
        """Use only the information that we need from CDS"""
        Tabls = []
        Band = [where(self.CDSTable['sed_filter'] == filt)[0] for filt in tableFilter]
        for bandinx, bandif in enumerate(Band):
            AVG, STD = self.CleanCDSTo(self.CDSTable[bandif])
            if isnan(AVG):
                continue
            else:
                Tabls.append([tableFilter[bandinx],
                              np.mean(self.CDSTable[bandif]['sed_freq'].to(u.micron,
                                                                           equivalencies=u.spectral())),
                              AVG*u.Jy, STD*u.Jy])
        return(Table(array(Tabls),
                     names=['Filter', 'Wave', 'Flux', 'F_er'],
                     dtype=('U32', 'float64', 'float64', 'float64')))

    def JoinTables(self):
        """Finally, we join the tables and remove those whose errors are too high"""
        self.TabFin = vstack([self.FiltersCDS(CDSFilters), self.FiltersNED(NEDFilters)])
        self.TabFin.remove_rows(where(self.TabFin['F_er']/self.TabFin['Flux'] >= 1))

Now we load the SMB-VCV sample and define the filters in CDS and NED that will be used to create the relations between them. 

In [8]:
Sample = Table.read('../Data/Final/VCV_SMB_otype.txt', format='ascii')

## Select filters/bands
Here we create a list for the bands we want to use from the CDS and NED

In [9]:
CDSFilters = ['GALEX:FUV', 'GALEX:NUV',
              "SDSS:u'", "SDSS:g'", "SDSS:r'", "SDSS:i'", "SDSS:z'",
              'SDSS:u', 'SDSS:g', 'SDSS:r', 'SDSS:i', 'SDSS:z',
              '2MASS:J', '2MASS:H', '2MASS:Ks',
              ':=3.6um', ':=4.5um', ':=5.8um', ':=8um',
              'WISE:W1', 'WISE:W2', 'WISE:W3', 'WISE:W4',
              'IRAS:12', 'IRAS:25', 'IRAS:60', 'IRAS:100',
              'Spitzer/MIPS:24', 'Spitzer/MIPS:70', 'Spitzer/MIPS:160',
              'Herschel/PACS:70', 'Herschel/PACS:100', 'Herschel/PACS:160',
              'Herschel/SPIRE:250', 'Herschel/SPIRE:350', 'Herschel/SPIRE:500',
              ':=250um', ':=350um', ':=500um',
              ':=1.4GHz', ':=21cm', ':=1.5GHz', ':=20cm', ':=5GHz', ':=6cm']

NEDFilters = [b'2-10 keV (XMM)', b'0.5-2 keV (XMM)',
              b'FUV (GALEX)', b'NUV (GALEX)',
              b'u (SDSS) AB', b'g (SDSS) AB', b'r (SDSS) AB', b'i (SDSS) AB', b'z (SDSS) AB',
              b'J (2MASS) AB', b'H (2MASS) AB', b'Ks (2MASS) AB',
              b'W1 (WISE)', b'W2 (WISE)', b'W3 (WISE)', b'W4 (WISE)',
              b'3.6 microns (IRAC)', b'4.5 microns (IRAC)', b'5.8 microns (IRAC)', b'8.0 microns (IRAC)',
              b'12 microns (IRAS)', b'25 microns (IRAS)', b'60 microns (IRAS)', b'100 microns (IRAS)',
              b'24 microns (MIPS)', b'70 microns (MIPS)', b'160 microns (MIPS)',
              b'70 microns (PACS)', b'100 microns (PACS)', b'160 microns (PACS)',
              b'250 microns (SPIRE)', b'350 microns (SPIRE)', b'500 microns (SPIRE)',
              b'4.89 GHz (VLA)', b'1.46 GHz (VLA)', b'1.4GHz']

## Save SEDs

With the information presented before and the selected bands, we look galaxy by galaxy for the SED in NED and CDS. Then, we clean the Photometry and write the final file per galaxy. 

In [35]:
# We show an example of the last galaxies from the positon 17723
for Galindx, Galaxy in enumerate(uniq(Sample['main_id'])[17723:]):
    if path.exists('../Data/Interim/SEDs/'+Galaxy+'_Phot.txt'):
        print(Galindx+17723)
        continue
    else:
        InitTabl = ObtainPhotometry(Galaxy)
        SED = CleanPhotometry(InitTabl.CDSTable, InitTabl.NEDTable).TabFin
        SED.write('../Data/Interim/SEDs/'+Galaxy+'_Phot.txt', format='ascii', overwrite='False')
        print(Galindx+17723)

17723
17724
17725
17726
17727
17728
17729
17730
17731
17732
17733
17734
17735
17736
17737
17738
17739
17740
17741
17742
17743
17744
17745
17746
17747
17748
17749
17750
17751
17752
17753
17754
17755
17756
17757
17758
17759
17760
17761
17762
17763
17764
17765
17766
17767
17768
17769
17770
17771
17772
17773
17774
17775
17776
17777
17778
17779
17780
17781
17782
17783
17784
17785
17786
17787
17788
17789
17790
17791
17792
17793
17794
17795
17796
17797
17798
17799
17800
17801
17802
17803
17804
17805
17806
17807
17808
17809
17810
17811
17812
17813
17814
17815
17816
17817
17818
17819
17820
17821
17822
17823
17824
17825
17826
17827
17828
17829
17830
17831
17832
17833
17834
17835
17836
17837
17838
17839
17840
17841
17842
17843
17844
17845
17846
17847
17848
17849
17850
17851
17852
17853
17854
17855
17856
17857
17858
17859
17860
17861
17862
17863
17864
17865
17866
17867
17868
17869
17870
17871
17872
17873
17874
17875
17876
17877
17878
17879
17880
17881
17882
17883
17884
17885
17886
17887
17888
1788

We create a .py script with the code available in this Jupyter notebook (located in [Additionals](../Additionals/3_Obtain_SEDs.py)) to meet the time requirements of the url request (Important if experimenting problems with retrieving times). Sometimes when there are some problems with the VOTables (when you lost connection and the saved file is truncated) is better to try to download the CDS file manually and add it into the folder. Keep in mind some limitations on the notebook server (i.e. iopub_msg_rate_limit)

##### Notebook info

In [10]:
%load_ext watermark
%watermark -a "Andres Ramos" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astroquery

Author: Andres Ramos

Python implementation: CPython
Python version       : 3.8.3
IPython version      : 7.16.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 3.10.0-1160.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astroquery: 0.4.1

logging   : 0.5.1.2
requests  : 2.24.0
matplotlib: 3.2.2
numpy     : 1.19.5
sys       : 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
re        : 2.2.1
json      : 2.0.9
astropy   : 4.2

Watermark: 2.1.0

